## Importing Packages

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd.variable import Variable
from torchvision import datasets, transforms
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import imageio
import torchvision.utils as vutils

## Loading the Rock/Paper/Scissor Dataset

In [9]:

train_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5], [0.5,0.5,0.5])])
train_dataset = datasets.ImageFolder(root='./data/rps', transform=train_transform)
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)


## Initializing the Weights

In [3]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        torch.nn.init.normal_(m.weight, 1.0, 0.02)
        torch.nn.init.zeros_(m.bias)    

## Creating the Generator & Discriminator Class

In [4]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
         
      
        self.label_conditioned_generator =nn.Sequential(nn.Embedding(3, 100),
                      nn.Linear(100, 16))
         
     
        self.latent =nn.Sequential(nn.Linear(100, 4*4*512),
                                   nn.LeakyReLU(0.2, inplace=True))
            
 
        self.model =nn.Sequential(nn.ConvTranspose2d(513, 64*8, 4, 2, 1, bias=False),
                      nn.BatchNorm2d(64*8, momentum=0.1,  eps=0.8),
                      nn.ReLU(True),
                      nn.ConvTranspose2d(64*8, 64*4, 4, 2, 1,bias=False),
                      nn.BatchNorm2d(64*4, momentum=0.1,  eps=0.8),
                      nn.ReLU(True), 
                      nn.ConvTranspose2d(64*4, 64*2, 4, 2, 1,bias=False),
                      nn.BatchNorm2d(64*2, momentum=0.1,  eps=0.8),
                      nn.ReLU(True), 
                      nn.ConvTranspose2d(64*2, 64*1, 4, 2, 1,bias=False),
                      nn.BatchNorm2d(64*1, momentum=0.1,  eps=0.8),
                      nn.ReLU(True), 
                      nn.ConvTranspose2d(64*1, 3, 4, 2, 1, bias=False),
                      nn.Tanh())
 
    def forward(self, inputs):
        noise_vector, label = inputs
        label_output = self.label_conditioned_generator(label)
        label_output = label_output.view(-1, 1, 4, 4)
        latent_output = self.latent(noise_vector)
        latent_output = latent_output.view(-1, 512,4,4)
        concat = torch.cat((latent_output, label_output), dim=1)
        image = self.model(concat)
        #print(image.size())
        return image

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
         
        self.label_condition_disc = nn.Sequential(nn.Embedding(3, 100),
                      nn.Linear(100, 3*128*128))       
        
        self.model = nn.Sequential(nn.Conv2d(3, 64, 4, 2, 1, bias=False),
                      nn.LeakyReLU(0.2, inplace=True),
                      nn.Conv2d(64, 64*2, 4, 3, 2, bias=False),
                      nn.BatchNorm2d(64*2, momentum=0.1,  eps=0.8),
                      nn.LeakyReLU(0.2, inplace=True),
                      nn.Conv2d(64*2, 64*4, 4, 3,2, bias=False),
                      nn.BatchNorm2d(64*4, momentum=0.1,  eps=0.8),
                      nn.LeakyReLU(0.2, inplace=True),
                      nn.Conv2d(64*4, 64*8, 4, 3, 2, bias=False),
                      nn.BatchNorm2d(64*8, momentum=0.1,  eps=0.8),
                      nn.LeakyReLU(0.2, inplace=True), 
                      nn.Flatten(),
                      nn.Dropout(0.4),
                      nn.Linear(9216, 1),
                      nn.Sigmoid()
                     )
 
    def forward(self, inputs):
        img, label = inputs
        label_output = self.label_condition_disc(label)
        label_output = label_output.view(-1, 3, 128, 128)
        concat = torch.cat((img, label_output), dim=-1)
        #print(concat.size())
        output = self.model(concat)
        return output

## Creating Objects, Optimizer, BCE Loss & Vectors

In [5]:
def noise(n, n_features=128):
    return Variable(torch.randn(n, n_features)).to(device)


device = 'cuda'

generator = Generator().to(device)
generator.apply(weights_init)
discriminator = Discriminator().to(device)
discriminator.apply(weights_init)

learning_rate = 0.0002
G_optimizer = optim.Adam(generator.parameters(), lr = learning_rate, betas=(0.5, 0.999))
D_optimizer = optim.Adam(discriminator.parameters(), lr = learning_rate, betas=(0.5, 0.999))

images = []
test_noise = noise(100)

adversarial_loss = nn.BCELoss()



## Calling the BCE Loss function

In [6]:
def generator_loss(fake_output, label):
    gen_loss = adversarial_loss(fake_output, label)
    #print(gen_loss)
    return gen_loss

def discriminator_loss(output, label):
    disc_loss = adversarial_loss(output, label)
    return disc_loss

## Training the Networks

In [7]:
num_epochs = 200
for epoch in range(1, num_epochs+1): 
    D_loss_list, G_loss_list = [], []
    g_error, d_error=0.0, 0.0
    print("Epoch :",epoch)
    for index, (real_images, labels) in enumerate(train_loader):
        D_optimizer.zero_grad()
        real_images = real_images.to(device)
        labels = labels.to(device)
        labels = labels.unsqueeze(1).long()
 
       
        real_target = Variable(torch.ones(real_images.size(0), 1).to(device))
        fake_target = Variable(torch.zeros(real_images.size(0), 1).to(device))
       
        D_real_loss = discriminator_loss(discriminator((real_images, labels)), real_target)
        # print(discriminator(real_images))
        #D_real_loss.backward()
     
        noise_vector = torch.randn(real_images.size(0), 100, device=device)  
        noise_vector = noise_vector.to(device)
         
        
        generated_image = generator((noise_vector, labels))
        output = discriminator((generated_image.detach(), labels))
        D_fake_loss = discriminator_loss(output,  fake_target)
 
     
        # train with fake
        #D_fake_loss.backward()
       
        D_total_loss = (D_real_loss + D_fake_loss) / 2
        D_loss_list.append(D_total_loss)
       
        D_total_loss.backward()
        D_optimizer.step()
 
        # Train generator with real labels
        G_optimizer.zero_grad()
        G_loss = generator_loss(discriminator((generated_image, labels)), real_target)
        G_loss_list.append(G_loss)
 
        G_loss.backward()
        G_optimizer.step()
        
        g_error += G_loss
        d_error += D_total_loss
        
        if index%15 ==0 :
            vutils.save_image(real_images, '%s/real_samples.png' % "./results", normalize = True)
            fake = generator((noise_vector,labels))
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("./results", epoch), normalize = True)
    print('Epoch {}: g_loss: {:.8f} d_loss: {:.8f}\r'.format(epoch, g_error/index, d_error/index))
    
    img = generator((noise_vector,labels)).cpu().detach()
    img = make_grid(img)
    images.append(img)
        
    
print('Training Finished')
torch.save(generator.state_dict(), 'Conditional-GAN.pth')

Epoch : 1
Epoch 1: g_loss: 2.54973888 d_loss: 0.30868986
Epoch : 2
Epoch 2: g_loss: 6.43261433 d_loss: 0.01482268
Epoch : 3
Epoch 3: g_loss: 10.39438534 d_loss: 0.11885827
Epoch : 4
Epoch 4: g_loss: 4.58300400 d_loss: 0.40138936
Epoch : 5
Epoch 5: g_loss: 3.26647854 d_loss: 0.33969256
Epoch : 6
Epoch 6: g_loss: 3.87531495 d_loss: 0.23192680
Epoch : 7
Epoch 7: g_loss: 4.78416252 d_loss: 0.25680047
Epoch : 8
Epoch 8: g_loss: 5.04929399 d_loss: 0.54201931
Epoch : 9
Epoch 9: g_loss: 3.38708711 d_loss: 0.33442551
Epoch : 10
Epoch 10: g_loss: 5.02778339 d_loss: 0.42029729
Epoch : 11
Epoch 11: g_loss: 3.41607261 d_loss: 0.43709859
Epoch : 12
Epoch 12: g_loss: 2.70365405 d_loss: 0.30947712
Epoch : 13
Epoch 13: g_loss: 2.88833117 d_loss: 0.56385010
Epoch : 14
Epoch 14: g_loss: 2.27438402 d_loss: 0.49697697
Epoch : 15
Epoch 15: g_loss: 4.06546736 d_loss: 0.51908034
Epoch : 16
Epoch 16: g_loss: 3.60778427 d_loss: 0.32007274
Epoch : 17
Epoch 17: g_loss: 3.14119387 d_loss: 0.61742479
Epoch : 18
Epo

Epoch 139: g_loss: 1.78549647 d_loss: 0.49017021
Epoch : 140
Epoch 140: g_loss: 1.83087373 d_loss: 0.48579440
Epoch : 141
Epoch 141: g_loss: 2.08775997 d_loss: 0.49087232
Epoch : 142
Epoch 142: g_loss: 1.88530016 d_loss: 0.47320977
Epoch : 143
Epoch 143: g_loss: 1.76153743 d_loss: 0.47236285
Epoch : 144
Epoch 144: g_loss: 2.04338264 d_loss: 0.53600895
Epoch : 145
Epoch 145: g_loss: 1.81437802 d_loss: 0.48033306
Epoch : 146
Epoch 146: g_loss: 1.91237593 d_loss: 0.48745930
Epoch : 147
Epoch 147: g_loss: 2.07507992 d_loss: 0.54505014
Epoch : 148
Epoch 148: g_loss: 1.90643990 d_loss: 0.44887039
Epoch : 149
Epoch 149: g_loss: 2.23041248 d_loss: 0.56925297
Epoch : 150
Epoch 150: g_loss: 2.32545400 d_loss: 0.50482839
Epoch : 151
Epoch 151: g_loss: 1.87914085 d_loss: 0.49598262
Epoch : 152
Epoch 152: g_loss: 1.82312703 d_loss: 0.48816794
Epoch : 153
Epoch 153: g_loss: 2.34805536 d_loss: 0.56274807
Epoch : 154
Epoch 154: g_loss: 1.83344483 d_loss: 0.47811356
Epoch : 155
Epoch 155: g_loss: 1.874

## Looping through the Images and creating a GIF 

In [57]:
frames = []
for i in range(1,len(images)+1):
    image = imageio.v2.imread(f'%s/fake_samples_epoch_%03d.png'% ("./results", i))
    frames.append(image[:500])
    print(f'%s/fake_samples_epoch_%03d.png'% ("./results", i))

./results/fake_samples_epoch_001.png
./results/fake_samples_epoch_002.png
./results/fake_samples_epoch_003.png
./results/fake_samples_epoch_004.png
./results/fake_samples_epoch_005.png
./results/fake_samples_epoch_006.png
./results/fake_samples_epoch_007.png
./results/fake_samples_epoch_008.png
./results/fake_samples_epoch_009.png
./results/fake_samples_epoch_010.png
./results/fake_samples_epoch_011.png
./results/fake_samples_epoch_012.png
./results/fake_samples_epoch_013.png
./results/fake_samples_epoch_014.png
./results/fake_samples_epoch_015.png
./results/fake_samples_epoch_016.png
./results/fake_samples_epoch_017.png
./results/fake_samples_epoch_018.png
./results/fake_samples_epoch_019.png
./results/fake_samples_epoch_020.png
./results/fake_samples_epoch_021.png
./results/fake_samples_epoch_022.png
./results/fake_samples_epoch_023.png
./results/fake_samples_epoch_024.png
./results/fake_samples_epoch_025.png
./results/fake_samples_epoch_026.png
./results/fake_samples_epoch_027.png
.

In [58]:
imageio.mimsave('./progress.gif', # output gif
                frames,          # array of input frames
                fps = 7)         # optional: frames per second